# AI Judge pre-processing - person borders

In [1]:
import numpy as np
import cv2 # pip install opencv-python

In [2]:
# Global variables
rectangles = []
current_rectangle = None
drawing = False
frame = None

def draw_rectangle(event, x, y, flags, params):
    global frame, rectangles, current_rectangle, drawing

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        current_rectangle = [(x, y)]

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            current_rectangle.append((x, y))

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        current_rectangle.append((x, y))
        rectangles.append(current_rectangle)
        current_rectangle = None

def main(video_path):
    global frame, rectangles

    cap = cv2.VideoCapture(video_path)

    cv2.namedWindow('Frame')
    cv2.setMouseCallback('Frame', draw_rectangle)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        for rect in rectangles:
            cv2.rectangle(frame, rect[0], rect[1], (0, 255, 0), 2)

        cv2.imshow('Frame', frame)

        key = cv2.waitKey(1)
        if key == ord('q'):  # Press 'q' to exit
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_path = 'videos/20240209_atelier_006.mp4'  # Change to your video file path
    main(video_path)

In [3]:
ord('q')

113

In [4]:
def framecount(video_path):
    """Shows the total framecount of a video"""
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    return total_frames

    # Rest of your code goes here

    cap.release()
    cv2.destroyAllWindows()

In [5]:
import math
def closest_point(original0, original1, new):
    """Returns the index (0 or 1) of the point"""
    # Calculate distances between the new tuple and the original tuples
    distance0 = math.sqrt((new[0] - original0[0])**2 + (new[1] - original0[1])**2)
    distance1 = math.sqrt((new[0] - original1[0])**2 + (new[1] - original1[1])**2)
    
    # Check which original tuple is closer and replace it with the new tuple
    return 0 if distance0 < distance1 else 1

In [20]:
def get_specific_frame(video_path, frame_number, size=1):
    """Shows a specific frame number"""
    cv2.namedWindow('Frame')

    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    
    rect_corners = []
    def redraw_frame_with_rectangle(frame, mousetype, x, y, rect_corners):
        if len(rect_corners) < 2:
            rect_corners.append((x,y))
        else:
            idx = closest_point(rect_corners[0], rect_corners[1], (x,y))
            rect_corners[idx] = (x,y)

        if len(rect_corners) == 1:
            x1 = (x,y)
            x22, x21 = frame.shape[0:2]
            x2 = (x21, x22)
        else:
            x1 = rect_corners[0]
            x2 = rect_corners[1]

        frame_w_rect = cv2.rectangle(frame.copy(), x1, x2, [0,255,0])
        cv2.imshow('Frame', frame_w_rect)
      

    ret, frame = cap.read()
    idx = 0
    cv2.setMouseCallback('Frame', lambda a, b, c, d, e: redraw_frame_with_rectangle(frame, a, b, c, rect_corners) if a == 1 else None)

    if ret:
        if size != 1:
            frame = cv2.resize(frame, None, fx=size, fy=size)
        cv2.imshow('Frame', frame)
        
        
        while cv2.waitKey(0) != ord('q'):
            pass
    else:
        print("Frame not found.")

    cap.release()
    cv2.destroyAllWindows()

    def switch_rect_corners_if_needed(rect_corners):
        if len(rect_corners) < 2:
            return [(0,0),(0,0)]
        idx = closest_point(rect_corners[0], rect_corners[1], (0,0))
        return [rect_corners[idx], rect_corners[1-idx]]
    
    return switch_rect_corners_if_needed(rect_corners)

In [21]:
video = "20240209_atelier_006.mp4"
video_path = 'videos/' + video  # Change to your video file path
image_path = 'images/' + video
framecount(video_path)

896

In [22]:
get_specific_frame(video_path, 10, 0.5)

[(0, 0), (0, 0)]

In [ ]:
# Label videos

In [23]:
import random
frame_rectangles = {}
def label_videos():
    for i in range(20):
        fid = random.randint(0, framecount(video_path))
        rectangle_coords = get_specific_frame(video_path, fid, 0.5)
        frame_rectangles[fid] = rectangle_coords

In [33]:
label_videos()

In [32]:
frame_rectangles

{810: [(0, 0), (0, 0)],
 754: [(23, 366), (220, 766)],
 22: [(0, 0), (0, 0)],
 185: [(309, 395), (120, 759)],
 571: [(238, 414), (102, 751)],
 893: [(0, 0), (0, 0)],
 229: [(84, 409), (261, 725)],
 805: [(0, 0), (0, 0)],
 259: [(316, 418), (122, 747)],
 464: [(241, 409), (89, 757)],
 799: [(0, 0), (0, 0)],
 852: [(0, 0), (0, 0)],
 456: [(54, 437), (222, 748)],
 326: [(281, 393), (99, 735)],
 675: [(30, 414), (232, 773)],
 816: [(0, 0), (0, 0)],
 395: [(80, 437), (278, 738)],
 183: [(307, 408), (110, 748)],
 288: [(252, 434), (56, 731)],
 752: [(36, 306), (223, 768)]}

In [36]:
ordered_dict = dict(sorted(frame_rectangles.items()))
ordered_dict

{22: [(0, 0), (0, 0)],
 44: [(0, 0), (0, 0)],
 47: [(0, 0), (0, 0)],
 68: [(273, 424), (2, 956)],
 109: [(262, 413), (95, 794)],
 156: [(314, 360), (107, 773)],
 165: [(288, 411), (111, 751)],
 183: [(108, 407), (313, 744)],
 185: [(309, 395), (120, 759)],
 202: [(259, 430), (145, 755)],
 229: [(84, 409), (261, 725)],
 259: [(316, 418), (122, 747)],
 267: [(291, 412), (136, 742)],
 273: [(253, 426), (150, 747)],
 288: [(258, 429), (55, 731)],
 315: [(280, 425), (90, 735)],
 326: [(281, 393), (99, 735)],
 395: [(85, 428), (281, 741)],
 456: [(54, 437), (222, 748)],
 464: [(241, 409), (89, 757)],
 504: [(266, 431), (73, 762)],
 571: [(238, 414), (102, 751)],
 607: [(46, 364), (240, 767)],
 634: [(255, 402), (41, 767)],
 675: [(30, 414), (232, 773)],
 729: [(262, 384), (21, 784)],
 752: [(36, 306), (223, 768)],
 754: [(23, 366), (220, 766)],
 799: [(0, 0), (0, 0)],
 805: [(0, 0), (0, 0)],
 810: [(0, 0), (0, 0)],
 816: [(0, 0), (0, 0)],
 845: [(0, 0), (0, 0)],
 852: [(0, 0), (0, 0)],
 876:

In [38]:
import pickle

with open('rectangles_atelier_006.pkl', 'wb') as f:
    pickle.dump(ordered_dict, f)
    f.close()